In [ ]:
hateXplain = pd.read_json("/home/mariannettrd/code/irismarechal1997/moder_ia/raw_data/230911_HateXplain.json")

hateXplain_inversed = hateXplain.transpose()

hateXplain=hateXplain_inversed


hateXplain["source"]="230911_HateXplain" # add source of the doc


hateXplain = hateXplain[["post_tokens", "annotators", "source"]].copy()

hateXplain.reset_index(inplace=True, drop=True)

hateXplain["offensive"]=0



In [ ]:
hateXplain.head()

In [ ]:
# for i in range(0,len(hateXplain)): 
#     hateXplain.iloc[i,-1]= hateXplain.iloc[i,1][0]["label"]

In [ ]:
hateXplain["offensive"]=hateXplain["annotators"].apply(lambda x:x[0]["label"]).apply(lambda x:0 if x =="normal" else 1)
hateXplain["offensive"]

In [ ]:
hateXplain

## ML for multilabelling

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import hamming_loss

In [ ]:
file_path = "/home/mariannettrd/code/irismarechal1997/moder_ia/data/data_classif.csv"
df = pd.read_csv(file_path)

In [ ]:
df.columns

In [ ]:
df.drop(columns="Unnamed: 0", inplace= True)


In [ ]:
df.drop(columns ="hate_speech_score", inplace = True)

In [ ]:
df

In [ ]:
dictrename = {'target_race': 'racism',
              'target_religion': 'religion',
              'target_origin': 'xenophobia',
              'target_gender_women':'misogyny',
              'target_gender_without_women':'transphobia',
              'target_sexuality': 'homophobia',
              'target_age': 'ageism',
              'target_disability':'validism'} 

In [ ]:
df.rename(columns={'target_race': 'racism',
              'target_religion': 'religion',
              'target_origin': 'xenophobia',
              'target_gender_women':'misogyny',
              'target_gender_without_women':'transphobia',
              'target_sexuality': 'homophobia',
              'target_age': 'ageism',
              'target_disability':'validism'}, inplace = True )

In [ ]:
df

In [ ]:
for row in ['racism', 'religion', 'xenophobia', 'misogyny', 'transphobia', 'homophobia', 'ageism', 'validism']:
    df[row] = df[row].replace({False: 0, True: 1})

In [ ]:
df

In [ ]:
def cleaning_table(data):
    data = data.drop_duplicates() # Remove duplicates
    data = data.dropna(subset=['offensive']) # Remove n.a. values in columns 'Label' => check column
    return data

In [ ]:

def cleaning_table(data):
    data = data.drop_duplicates() # Remove duplicates
    data = data.dropna(subset=['text']) # Remove n.a. values in columns 'Label' => check column
    return data

# Note: no need to Scale the features, Encode features, Perform cyclical engineering

def cleaning_text(sentence: str) -> str:

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    #remove words
    removed = ["user", "rt"]
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in removed
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]

    cleaned_sentence = ' '.join(word for word in lemmatized)

    return cleaned_sentence

In [ ]:
data_processed = cleaning_table(df)

In [ ]:
data_processed["text_processed"] = data_processed["text"].apply(cleaning_text)

In [ ]:
data_processed

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss

ModelsPerformance = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    hamLoss = hamming_loss(test_labels, predictions)
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    ModelsPerformance[modelName] = micro_f1

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

In [ ]:
# Pipeline vectorizer + LinearSVC
pipeline = make_pipeline(
    TfidfVectorizer(), 
    OneVsRestClassifier(LinearSVC(), n_jobs=-1))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
svmPreds = pipeline.predict(X_test)
metricsReport("SVC Sq. Hinge Loss", y_test, svmPreds)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
pipeline = make_pipeline(
    TfidfVectorizer(), 
    OneVsRestClassifier(MultinomialNB()))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
svmPreds = pipeline.predict(X_test)
metricsReport("Multinomial NB", y_test, svmPreds)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#import tensorflow as tf
#import keras
from tensorflow.keras.utils import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
# import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Split into training and testing data
X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(8, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])


es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=5,
        batch_size=128,
        callbacks=[es]
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
from keras.layers import Embedding

# Split into training and testing data
X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

# create the model
max_review_length = 600

embedding_vector_length = 32
cnn_model = Sequential()


cnn_model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size))   
cnn_model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(layers.MaxPooling1D(pool_size=2))
cnn_model.add(layers.LSTM(100))
cnn_model.add(layers.Dense(8, activation='sigmoid'))

# Students will be ending their code here

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(cnn_model.summary())

# Change the number of epochs and the batch size depending on the RAM Size

es = EarlyStopping(patience=4, restore_best_weights=True)

history_c =model.fit(X_train_pad, y_train,
        epochs=5,
        batch_size=128,
        callbacks=[es]
        )
res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
train_predictions = model.predict(X_train_pad)
test_predictions = model.predict(X_test_pad)

In [ ]:
X_train_pad

## One model

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file_path = "../data/labelling_dataset_v1.csv"
df = pd.read_csv(file_path)

In [ ]:
df.head()

In [ ]:
labelled_data = pd.read_csv("/home/mariannettrd/code/irismarechal1997/moder_ia/data/labelling_dataset_v1.csv")
labelled_data.head()

In [ ]:
label = np.zeros(56515, dtype=np.int8 )
list_1 = label.tolist()
labelled_data.insert(1,"non-offensive",list_1)

In [ ]:
labelled_data.head()

In [ ]:
val_df=pd.DataFrame([
    {"racism":len(labelled_data[labelled_data["racism"]==1]),
       "religion":len(labelled_data[labelled_data["religion"]==1]),
       "xenophobia":len(labelled_data[labelled_data["xenophobia"]==1]),
       "misogyny":len(labelled_data[labelled_data["misogyny"]==1]),
     "non-offensive":len(labelled_data[labelled_data["non-offensive"]==1]),
       "transphobia":len(labelled_data[labelled_data["transphobia"]==1]),
       "validism":len(labelled_data[labelled_data["homophobia"]==1])}])
val_df, val_df.iloc[0].plot(kind="bar")

In [ ]:
df = pd.read_csv("/home/mariannettrd/code/irismarechal1997/moder_ia/data/processed_dataset_v1.csv")

In [ ]:
df.head()

In [ ]:
#select number of tweets
number_rows = 10_000

#filter on non-offensive
df2 = df[df["offensive"]==0]
sample_df=df2.sample(number_rows)

#select right columns to keep
selected_features = ["text","offensive", "text_processed"]
new_df = sample_df[selected_features].copy()
label = np.zeros(number_rows, dtype=np.int8 )
list_2 = label.tolist()


In [ ]:
new_df.head()

In [ ]:
labelled_data.head()
labelled_data.columns

In [ ]:
label = np.zeros(number_rows, dtype=np.int8 )
list_2 = label.tolist()

In [ ]:
new_df

In [ ]:
# Create a new DataFrame based on sample_df
new_df = sample_df[selected_features].copy()

# Define the columns you want to insert
# columns = ['non-offensive', 'racism', 'religion', 'xenophobia', 'misogyny', 'transphobia', 'homophobia']
columns= ['homophobia', 'transphobia', 'misogyny', 'xenophobia', 'religion', 'racism', 'non-offensive']
# Iterate through the columns and insert them into new_df
for col in columns:
    new_df.insert(1, col, list_2)

new_df["non-offensive"]=1
new_df.drop(columns="offensive", inplace = True)

In [ ]:

new_df

In [ ]:
labelled_data.head()

In [ ]:
data_set= pd.concat([labelled_data, new_df])

In [ ]:
data_set.head()

In [ ]:
# Split into training and testing data
X = data_set["text"]
y = data_set.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(7, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])


es = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=150,
        batch_size=128,
        callbacks=[es], validation_split=0.3
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

3. Model Iris avec Hamming loss 

In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss

import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import pad_sequences

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from keras.layers import Embedding


In [8]:

labelled_data = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
label = np.zeros(56515, dtype=np.int8 )
list_1 = label.tolist()
labelled_data.insert(1,"non-offensive",list_1)

df = pd.read_csv("../data/"+"processed_dataset_v1.csv")


#select number of tweets
number_rows = 10_000

#filter on non-offensive
df2 = df[df["offensive"]==0]
sample_df=df2.sample(number_rows)

#select right columns to keep
selected_features = ["text","offensive", "text_processed"]
new_df = sample_df[selected_features].copy()
label = np.zeros(number_rows, dtype=np.int8 )
list_2 = label.tolist()

# Define the columns you want to insert
columns= ['homophobia', 'transphobia', 'misogyny', 'xenophobia', 'religion', 'racism', 'non-offensive']

# Iterate through the columns and insert them into new_df
for col in columns:
    new_df.insert(1, col, list_2)
new_df["non-offensive"]=1
new_df.drop(columns="offensive", inplace = True)

data_set= pd.concat([labelled_data, new_df])

# Split into training and testing data
X = data_set["text"]
y = data_set.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(7, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])


es = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=50,
        batch_size=128,
        callbacks=[es], validation_split=0.3
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          2932850   
                                                                 
 gru_2 (GRU)                 (None, None, 20)          4320      
                                                                 
 gru_3 (GRU)                 (None, 20)                2520      
                                                                 
 dense_1 (Dense)             (None, 7)                 147       
                                                                 
Total params: 2939837 (11.21 MB)
Trainable params: 2939837 (11.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
273/273 [==============================] - 53s 162ms/step - loss: 0.4854 - accuracy: 0.2573 - val_loss: 0.4581 - val_accuracy: 0.2895
E

In [ ]:
#model=full_model_classif()

In [14]:
y_pred=model.predict(X_test_pad)
y_pred

520/520 [==============================] - 12s 20ms/step


array([[0.0555336 , 0.32921416, 0.00551936, ..., 0.47810352, 0.07550936,
        0.01757518],
       [0.02533962, 0.01991368, 0.02846691, ..., 0.16541786, 0.42184418,
        0.80412364],
       [0.05737745, 0.07361521, 0.00563814, ..., 0.5634848 , 0.14435072,
        0.06098539],
       ...,
       [0.9931581 , 0.00838253, 0.00795216, ..., 0.00990019, 0.0029719 ,
        0.01034239],
       [0.92378044, 0.00950552, 0.06980921, ..., 0.00542451, 0.00521516,
        0.02108896],
       [0.0600859 , 0.04420226, 0.33671063, ..., 0.00941777, 0.06304487,
        0.16687848]], dtype=float32)

In [15]:
import tensorflow as tf

y_true=y_test
y_pred=y_pred

def hamming_loss(y_true, y_pred, threshold=0.5):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    
    # Apply the threshold to the predicted values
    y_pred = tf.cast(y_pred > threshold, dtype=tf.float32)
    
    # Calculate the Hamming Loss
    epsilon = 1e-7  # Small epsilon value
    hamming_loss = 1.0 - tf.reduce_mean(tf.reduce_sum(y_true * y_pred, axis=1) / (tf.reduce_sum(y_true + y_pred - y_true * y_pred, axis=1) + epsilon))
    
    return hamming_loss.numpy()

In [16]:


h1 = hamming_loss(y_true, y_pred)
h1


#metric = metrics.HammingLoss(mode='multilabel', threshold=0.6)


#metric.update_state(y_true,y_pred)
#metric.result().numpy()



0.48068637

In [ ]:
type(y_true)

In [ ]:
full_model_classif()

In [17]:
data_processed = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
# Split into training and testing data
X = data_processed["text_processed"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(6, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=150,
        batch_size=128,
        callbacks=[es], validation_split=0.3
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
print('Testing loss \t', res[0]*100)
print('Testing accuracy ', res[1]*100)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          1958700   
                                                                 
 gru_4 (GRU)                 (None, None, 20)          4320      
                                                                 
 gru_5 (GRU)                 (None, 20)                2520      
                                                                 
 dense_2 (Dense)             (None, 6)                 126       
                                                                 
Total params: 1965666 (7.50 MB)
Trainable params: 1965666 (7.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/150
232/232 [==============================] - 27s 83ms/step - loss: 0.5297 - accuracy: 0.3490 - val_loss: 0.5163 - val_accuracy: 0.3509
Epo

In [18]:
y_true = y_test
y_pred=model.predict(X_test_pad)

h2 = hamming_loss(y_true, y_pred)
h2

442/442 [==============================] - 7s 11ms/step


0.4625392

In [23]:
data_processed = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
# Split into training and testing data
X = data_processed["text_processed"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

# create the model
model = Sequential()

model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.LSTM(100))
model.add(layers.Dense(6, activation='sigmoid'))

# Students will be ending their code here

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Change the number of epochs and the batch size depending on the RAM Size

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
    epochs=150,
    batch_size=128,
    callbacks=[es], validation_split=0.3
)
res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
print('Testing loss \t', res[0]*100)
print('Testing accuracy ', res[1]*100)


FileNotFoundError: [Errno 2] No such file or directory: '../data/labelling_dataset_v1.csv'

In [ ]:
y_true = y_test
y_pred=model.predict(X_test_pad)

h3 = hamming_loss(y_true, y_pred)
h3